[View source on GitHub]: https://github.com/wadmp/wadmp.github.io/blob/master/jupyter_notebooks/move_device.ipynb
[Notebook Viewer]: https://nbviewer.jupyter.org/github/wadmp/wadmp.github.io/blob/master/jupyter_notebooks/move_device.ipynb
[Run in binder]: https://mybinder.org/v2/gh/wadmp/wadmp.github.io/master?filepath=jupyter_notebooks%2Fmove_device.ipynb
[Run in Google Colab]: https://colab.research.google.com/github/wadmp/wadmp.github.io/blob/master/jupyter_notebooks/move_device.ipynb

| [![GitHub logo](https://raw.githubusercontent.com/wadmp/wadmp.github.io/master/images/github_logo.png)][View source on GitHub] | [![Jupyter logo](https://raw.githubusercontent.com/wadmp/wadmp.github.io/master/images/jupyter_logo.png)][Notebook Viewer] | [![binder logo](https://raw.githubusercontent.com/wadmp/wadmp.github.io/master/images/binder_logo.png)][Run in binder] | [![Colab logo](https://raw.githubusercontent.com/wadmp/wadmp.github.io/master/images/colab_logo.png)][Run in Google Colab] |
|:---------------------:|:---------------:|:-------------:|:-------------------:|
| [View source on GitHub] | [Notebook Viewer] | [Run in binder] | [Run in Google Colab] |

## Introduction
This notebook allows you to move a device from one WebAccess/DMP Management Server ("Server 1") to another WebAccess/DMP Management Server ("Server 2").

It does several things:
1. Create the device on Server 2; (if it is there already, that is not a problem)
2. Configure the TLS "trust store" on Server 2 to trust devices that present a certificate that was signed by Server 1;
3. Configure the Bootstrap Server on Server 1 to direct the device to Server 2;
4. Trigger the device to Bootstrap again.

### Requirements
* You need to have an existing user account on *both* WA/DMP servers.
* The device in question must exist on Server 1.
  * It does not necessarily need to be currently online on Server 1, but if it is not online then obviously you won't be able to confirm if the script has worked!
* On Server 1, you must have the "Device Management Server" permission.
* On Server 2, you must be a SysAdmin!

### Usage
Run the cells, either one at a time (Shift-Enter is your friend!), or all at once.

When prompted, enter the required User Input.

> Note that when entering the URL of each WA/DMP server, you ahould use the URL that you use in your web browser,
> when viewing the "User Interface" of WebAccess/DMP. e.g. https://staging.wadmp.com, or https://wadmp.com.
> The URL for the corresponding API gateway will be calculated automatically: https://gateway.staging.wadmp.com, or https://gateway.wadmp.com.

## Setup

In [ ]:
%%capture

# Install packages in the current Jupyter kernel
import sys
!{sys.executable} -m pip install requests
!{sys.executable} -m pip install cryptography

import os
from urllib.parse import urlparse, urlunparse
import requests
import json
import base64
import re
from cryptography import x509
from cryptography.x509.oid import NameOID
from cryptography.hazmat.backends import default_backend

## Global variables

In [ ]:
BASE_PATH = 'api'
SESSION1 = requests.Session()  # Use one HTTPS session for all API calls to Server 1
SESSION2 = requests.Session()  # Use one HTTPS session for all API calls to Server 2

## Functions to be used later

In [ ]:
def login(server, username, password):
    """Login to the system, and return a token
    """
    if server == 1:
        base_url = GATEWAY1
        session = SESSION1
    elif server == 2:
        base_url = GATEWAY2
        session = SESSION2
    
    url = f"{base_url}/public/auth/connect/token"
    credentials = {'username': username, 'password': password, 'client_id': 'python', 'grant_type': 'password'}
    print(f"Sending POST request to {url} with:\n"
          f"    credentials={credentials}\n")
    response = session.post(url, data=credentials)

    print(response.status_code)
    try:
        print(json.dumps(response.json(), indent=4, sort_keys=True))
    except ValueError:
        print(response.text)

    if response.status_code == requests.codes['ok']:
        return response.json()["access_token"]
    else:
        print("Failed to login!")
        sys.exit(1)


def get_devices(server, page_number, page_size, name):
    """Retrieves one page of the list of your devices.
       name = Optional filter for device’s alias.
    """
    if server == 1:
        base_url = GATEWAY1
        session = SESSION1
    elif server == 2:
        base_url = GATEWAY2
        session = SESSION2
    
    url = f"{base_url}/{BASE_PATH}/management/devices"

    # The only REQUIRED query parameters are page and pageSize
    print(f"Sending GET request to {url} with:\n"
          f"    name={name}\n"
          f"    page={page_number}\n"
          f"    pageSize={page_size}")
    query = {'name': name, 'page': page_number, 'pageSize': page_size}
    response = session.get(url, params=query)

    print(response.status_code)
    try:
        print(json.dumps(response.json(), indent=4, sort_keys=True))
    except ValueError:
        print(response.text)
    
    if response.status_code == requests.codes['ok']:
        return response.json()['data']
    else:
        print(f"Failed to retrieve page {page_number} of devices!")
        return None


def create_device(server, model=None):
    """Create a device in the system.
    """
    if server == 1:
        base_url = GATEWAY1
        session = SESSION1
    elif server == 2:
        base_url = GATEWAY2
        session = SESSION2
    
    url = f"{base_url}/{BASE_PATH}/identity/devices"
    print(f"Sending POST request to {url} with:\n"
          f"    model={model}\n")
    response = session.post(url, json=model)

    print(response.status_code)
    try:
        print(json.dumps(response.json(), indent=4, sort_keys=True))
    except ValueError:
        print(response.text)

    if response.status_code == requests.codes['ok']:
        return response.json()['data']['id']
    else:
        print.error("Failed to create device!")
        return None


def get_ms(server, Name=None, Address=None):
    """Gets the list of Management Server definition in the system.
    """
    if server == 1:
        base_url = GATEWAY1
        session = SESSION1
    elif server == 2:
        base_url = GATEWAY2
        session = SESSION2
    
    url = f"{base_url}/{BASE_PATH}/bootstrap"
    print(f"Sending GET request to {url} with:\n"
          f"    Name={Name}\n"
          f"    Address={Address}\n")
    query = {'Name': Name, 'Address': Address}
    response = session.get(url, params=query)

    print(response.status_code)
    try:
        print(json.dumps(response.json(), indent=4, sort_keys=True))
    except ValueError:
        print(response.text)

    if response.status_code == requests.codes['ok']:
        return response.json()['data']
    else:
        print("Failed to retrieve the list of Management Servers!")
        return None


def create_ms(server, name, address, port, company_id):
    """Creates a new Management Server definition in the system.
    
       name: is not important: it serves as a label or reminder for the user only.
       address: is the IP address or DNS name of the Management Server.
       port: is the TCP port number of the Management Server. (Usually 8883, which is the standard port number for MQTT over TLS.)
       company_id: is your company ID in the system.
    """
    if server == 1:
        base_url = GATEWAY1
        session = SESSION1
    elif server == 2:
        base_url = GATEWAY2
        session = SESSION2
    
    url = f"{base_url}/{BASE_PATH}/bootstrap"
    print(f"Sending POST request to {url} with:\n"
          f"    name={name}\n"
          f"    address={address}\n"
          f"    port={port}\n"
          f"    company_id={company_id}\n")
    body = {'name': name, 'address': address, 'port': port, 'company_id': company_id}
    response = session.get(url, json=body)

    print(response.status_code)
    try:
        print(json.dumps(response.json(), indent=4, sort_keys=True))
    except ValueError:
        print(response.text)

    if response.status_code == requests.codes['ok']:
        return response.json()['data'][0]['id']
    else:
        print("Failed to create new MS definition!")
        return None


def use_ms(server, mac, ms_id):
    """Configure a device to use a new Management Server.
       This will only take effect when the device Bootstraps again.
    """
    if server == 1:
        base_url = GATEWAY1
        session = SESSION1
    elif server == 2:
        base_url = GATEWAY2
        session = SESSION2
    
    url = f"{base_url}/{BASE_PATH}/management/devices/{mac}/bootstrap-server/{ms_id}"
    print(f"Sending PUT request to {url}\n")
    response = session.put(url)

    print(response.status_code)
    try:
        print(json.dumps(response.json(), indent=4, sort_keys=True))
    except ValueError:
        print(response.text)

    if response.status_code == requests.codes['ok']:
        return response.json()['success']
    else:
        print("Failed to configure device to use MS!")
        return None


def trigger_bootstrap(server, mac):
    """Sends a command to trigger bootstrap process on a device.
    """
    if server == 1:
        base_url = GATEWAY1
        session = SESSION1
    elif server == 2:
        base_url = GATEWAY2
        session = SESSION2
    
    url = f"{base_url}/{BASE_PATH}/management/devices/{mac}/commands/trigger-bootstrap"
    print(f"Sending POST request to {url}\n")
    response = session.post(url)

    print(response.status_code)
    try:
        print(json.dumps(response.json(), indent=4, sort_keys=True))
    except ValueError:
        print(response.text)

    if response.status_code == requests.codes['ok']:
        return response.json()['success']
    else:
        print.error("Failed to trigger bootstrap!")
        return None


def get_certs(server):
    """Gets the device CA certificates in the trust store of the Management Server.
       The returned value is a Base64-encoded string of all of the trusted CA certs, in PEM format.
    """
    if server == 1:
        base_url = GATEWAY1
        session = SESSION1
    elif server == 2:
        base_url = GATEWAY2
        session = SESSION2
    
    url = f"{base_url}/{BASE_PATH}/certs"
    print(f"Sending GET request to {url}\n")
    response = session.get(url)

    print(response.status_code)
    try:
        print(json.dumps(response.json(), indent=4, sort_keys=True))
    except ValueError:
        print(response.text)

    if response.status_code == requests.codes['ok']:
        return response.json()['data']['certs']
    else:
        print("Failed to retrieve the certificates!")
        return None


def put_certs(server, certs):
    """Replaces the device CA certificates in the trust store of the Management Server.
       certs = Base64-encoded string, comprising ALL trusted CA certificates in PEM format.
    """
    if server == 1:
        base_url = GATEWAY1
        session = SESSION1
    elif server == 2:
        base_url = GATEWAY2
        session = SESSION2
    
    url = f"{base_url}/{BASE_PATH}/certs"
    print(f"Sending PUT request to {url} with:\n"
          f"    certs={certs}\n")
    body = {'certs': certs}
    response = session.put(url, json=body)

    print(response.status_code)
    try:
        print(json.dumps(response.json(), indent=4, sort_keys=True))
    except ValueError:
        print(response.text)

    if response.status_code == requests.codes['ok']:
        return response.json()['success']
    else:
        print("Failed to edit MS definition!")
        return None

## User input

In [ ]:
device_name = input("Enter device alias or MAC address:")

SERVER1 = input("Enter the URL of Server 1 (e.g. 'https://staging.wadmp.com'):")

USERNAME1 = input("Username on Server 1:")
PASSWORD1 = input("Password on Server 1:")

# If the URL for the UI is "https://staging.wadmp.com", the URL for the API Gateway is "https://gateway.staging.wadmp.com"
url = urlparse(SERVER1)
new_url = f"gateway.{url.netloc}"
GATEWAY1 = urlunparse(url._replace(netloc=new_url))

# If the URL for the UI is "https://staging.wadmp.com", the URL for the Management Server is "management.staging.wadmp.com"
MS1 = f"management.{url.netloc}"

SERVER2 = input("Enter the URL of Server 2 (where you want the device to connect to next):")
USERNAME2 = input("Username of SysAdmin on Server 2:")
PASSWORD2 = input("Password on Server 2:")

# If the URL for the UI is "https://wadmp.com", the URL for the API Gateway is "https://staging.wadmp.com"
url = urlparse(SERVER2)
new_url = f"gateway.{url.netloc}"
GATEWAY2 = urlunparse(url._replace(netloc=new_url))

# If the URL for the UI is "https://wadmp.com", the URL for the Management Server is "management.wadmp.com"
MS2 = f"management.{url.netloc}"

## Login to Server 1

In [ ]:
user1_token = login(1, USERNAME1, PASSWORD1)
SESSION1.headers.update({'Authorization': f'Bearer {user1_token}'})

## Check that the device exists on Server 1

In [ ]:
device = get_devices(1, 1, 10, device_name)[0]

## Login to Server 2

In [ ]:
user2_token = login(2, USERNAME2, PASSWORD2)
SESSION2.headers.update({'Authorization': f'Bearer {user2_token}'})

## Create the device on Server 2

In [ ]:
# Check if it exists already
try:
    device2 = get_devices(2, 1, 10, device_name)[0]
except IndexError:
    print(f"Device {device_name} does not exist on Server 2. Creating it ...")
    model = {
      "alias": device['alias'],
      "serial_number": device['serial_number'],
      "mac_address": device['mac_address'],
      "device_type_id": device['device_type']['type_id'],
      "order_code": device['order_code'],
      "imei": device['imei']
    }
    create_device(2, model=model)
else:
    print(f"Device {device_name} already exists on Server 2")

## Configure the Trust Store on Server 2
We take a conservative approach: any certificates in Server 1 that are NOT in Server 2 will be added to Server 2

First, check what certs are present in Server 2:

In [ ]:
certs2_b64 = get_certs(2)
certs2_pem = base64.b64decode(certs2_b64).decode('utf-8')  # b64decode() returns a "bytes-like" object

pattern = r"(-----BEGIN CERTIFICATE-----[A-Za-z0-9+/=\s]+-----END CERTIFICATE-----)"

certs2_list = re.findall(pattern, certs2_pem, re.MULTILINE)
certs2_dict = {}
print(f"{MS2} has {len(certs2_list)} certs:")
for cert_pem in certs2_list:
    # load_pem_x509_certificate() takes a "bytes-like" object
    cert_object = x509.load_pem_x509_certificate(cert_pem.encode('utf-8'), default_backend())
    print(f"   - {cert_object.subject.get_attributes_for_oid(NameOID.COMMON_NAME)[0].value}")
    certs2_dict[cert_object.subject] = cert_pem

Now, check what certs are present in Server 1:

In [ ]:
certs1_b64 = get_certs(1)
certs1_pem = base64.b64decode(certs1_b64).decode('utf-8')
new_certs2_pem = certs2_pem  # We will append to this string
update_required = False

certs1_list = re.findall(pattern, certs1_pem, re.MULTILINE)
print(f"{MS1} has {len(certs1_list)} certs:")
for cert_pem in certs1_list:
    cert_object = x509.load_pem_x509_certificate(cert_pem.encode('utf-8'), default_backend())
    print(f"   - {cert_object.subject.get_attributes_for_oid(NameOID.COMMON_NAME)[0].value}")
    if cert_object.subject in certs2_dict:
        print(f"     Also in {MS2}")
    else:
        print(f"     Not in {MS2} ... adding")
        new_certs2_pem += "\n" + cert_pem
        update_required = True

if update_required:
    new_certs2_b64 = base64.b64encode(new_certs2_pem.encode('utf-8')).decode('utf-8')  # b64encode() takes a "bytes-like" object and returns bytes
    put_certs(2, new_certs2_b64)

## Inform Server 1 that Server 2 is available as a new Management Server

In [ ]:
ms_definitions = get_ms(1, Address=MS2)

if len(ms_definitions) == 0:
    print(f"We need to create {MS2} as a new MS definition on {MS1}")
    ms_id = create_ms(1, 'Server 2', MS2, 8883)
else:
    print(f"{MS2} is already defined on {MS1}")
    ms_definition = ms_definitions[0]
    ms_id = ms_definition['id']

## Configure the device to use the new Management Server definition

In [ ]:
use_ms(1, device['mac_address'], ms_id)

## Trigger the device to bootstrap again

In [ ]:
trigger_bootstrap(1, device['mac_address'])